Import necessary library

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import urllib.request
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

Getting information about the events on the site from the api

In [3]:
def data_scrap(url: str):
    url = url
    response = requests.get(url)

    if response.status_code == 200:

        soup = BeautifulSoup(response.text, "html.parser")

        h3_tags_title = soup.find_all("h3", class_="blog_post_title my-2")
        h3_tags_location = soup.find_all("div", class_="blog_post_title my-2")
        h3_tags_date = soup.find_all("div", class_="theater-date my-2")

        titles = []
        location = []
        date = []

        for title in h3_tags_title:
            if title.text.strip():
                titles.append(title.text.strip())

        df_title = pd.DataFrame({"Titles": titles})

        for loc in h3_tags_location:
            if loc.text.strip():
                location.append(loc.text.strip())

        df_location = pd.DataFrame({"Titles": location})

        for dt in h3_tags_date:
            if dt.text.strip():
                date.append(dt.text.strip())

        df_date = pd.DataFrame({"Titles": date})

    return df_title, df_date, df_location

In [4]:
# load events
df_title, df_date, df_location = data_scrap("https://www.6234.ir/")

Getting information about the user buy and Interaction history on the site from the api

In [5]:
def user_buy_iteraction_from_api(buy_api: str, iter_api: str):
    buy_link = buy_api
    iter_link = iter_api
    urllib.request.urlretrieve(iter_link, "log.xlsx")
    iter_history = pd.read_excel("log.xlsx")
    urllib.request.urlretrieve(buy_link, "visitor.xlsx")
    buy_history = pd.read_excel("visitor.xlsx")

    return iter_history, buy_history

In [6]:
# load user buy and interaction from api

interaction, buy_history = user_buy_iteraction_from_api(
    "https://6234.ir/api/ticket?token=apiqazxcvbnm&ofDate=1402/08/20&toDate=1402/12/29",
    "https://6234.ir/api/log?token=apiqazxcvbnm&ofDate=1402/08/20&toDate=1402/12/29",
)

Data preparation

In [7]:
interaction.head()

,نام و نام خانوادگی,شماره موبایل,بازدید,زمان تعامل(تانیه),آی پی,تاریخ,تگ ها,توضیحات
0,ناشناس,NaN,صفحه اصلی,0.0,83.123.96.25,1402/12/28 21:17:53,NaN,NaN
1,ناشناس,NaN,صفحه اصلی,0.0,190.212.57.90,1402/12/28 21:17:42,NaN,NaN
2,ناشناس,NaN,صفحه اصلی,0.0,43.135.149.154,1402/12/28 19:33:12,NaN,NaN
3,ناشناس,NaN,صفحه اصلی,0.0,104.233.15.70,1402/12/28 19:11:35,NaN,NaN
4,ناشناس,NaN,صفحه اصلی,0.0,129.226.146.179,1402/12/28 18:45:03,NaN,NaN


Visiting is important here and we will fill the empty space

In [8]:
interaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30204 entries, 0 to 30203
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   نام و نام خانوادگی  30186 non-null  object 
 1   شماره موبایل        134 non-null    float64
 2   بازدید              30176 non-null  object 
 3   زمان تعامل(تانیه)   30204 non-null  float64
 4   آی پی               30204 non-null  object 
 5   تاریخ               30204 non-null  object 
 6   تگ ها               136 non-null    object 
 7   توضیحات             4644 non-null   object 
dtypes: float64(2), object(6)
memory usage: 1.8+ MB


In [9]:
interaction.isnull().sum()

نام و نام خانوادگی       18
شماره موبایل          30070
بازدید                   28
زمان تعامل(تانیه)         0
آی پی                     0
تاریخ                     0
تگ ها                 30068
توضیحات               25560
dtype: int64

In [10]:
# fill na
interaction["بازدید"] = interaction["بازدید"].fillna("ffill")
interaction["نام و نام خانوادگی"] = interaction["نام و نام خانوادگی"].fillna("none")
interaction["شماره موبایل"] = interaction["شماره موبایل"].fillna("none")

In [11]:
# Drop صفحه اصلی from بازدید columns
interaction = interaction[interaction["بازدید"] != "صفحه اصلی"]

In [12]:
buy_history.head()

,شماره بلیط,نام و نام خانوادگی,شماره موبایل,ایمیل,تاریخ تولد,سن,جنسیت,استان سکونت,شهر سکونت,تحصیلات,...,صندلی,استان,شهر,سالن,تهیه کننده,صحنه گردان,رویداد,تاریخ سانس,ساعت سانس,تاریخ ثبت
0,1600,سفردوست,9101758962,NaN,1354/01/08,49,male,تهران,تهران,NaN,...,NaN,NaN,NaN,ترکیه,NaN,NaN,پارک امیرگان ( Test ),1 فروردین 1403,NaN,1402/08/20 18:12:26
1,1601,امیر نبی پور,9117803218,nabipour63@gmail.com,1363/01/01,40,male,تهران,تهران,کارشناسی ارشد,...,NaN,NaN,NaN,NaN,موسسه شکوه موسیقی ایرانیان,NaN,مسابقه آنلاین,1 فروردین 1403,20:00,1402/08/26 22:46:13
2,1602,سفردوست,9101758962,NaN,1354/01/08,49,male,تهران,تهران,NaN,...,NaN,NaN,NaN,ترکیه,NaN,NaN,پارک امیرگان ( Test ),1 فروردین 1403,NaN,1402/09/01 11:24:12
3,1603,سفردوست,9101758962,NaN,1354/01/08,49,male,تهران,تهران,NaN,...,NaN,NaN,NaN,NaN,موسسه شکوه موسیقی ایرانیان,NaN,مسابقه آنلاین,1 فروردین 1403,20:00,1402/09/03 16:17:00
4,1604,سفردوست,9101758962,NaN,1354/01/08,49,male,تهران,تهران,NaN,...,NaN,NaN,NaN,NaN,موسسه شکوه موسیقی ایرانیان,NaN,مسابقه آنلاین,1 فروردین 1403,20:00,1402/09/03 16:24:22


In [13]:
# merge all df
def merge_df(
    title_scrap: pd.DataFrame,
    location_scrap: pd.DataFrame,
    date_scrap: pd.DataFrame,
    itraction,
    buy_history,
):

    merge_df = pd.DataFrame(
        {
            "Titles": title_scrap["Titles"],
            "Location": location_scrap["Titles"],
            "Date": date_scrap["Titles"],
        }
    )

    merge_df = pd.concat(
        [merge_df["Titles"], itraction["بازدید"], buy_history["رویداد"]]
    ).reset_index()

    merge_df.columns = ["index", "Titles"]

    return merge_df

In [14]:
# use merge df func to merge all df

merged_df = merge_df(
    title_scrap=df_title,
    location_scrap=df_location,
    date_scrap=df_date,
    itraction=interaction,
    buy_history=buy_history,
)
merged_df.head()

,index,Titles
0,0,کنسرت تست ( بدون انتخاب صندلی )
1,1,کنسرت تست ( با انتخاب صندلی )
2,2,کنگره بین‌المللی جامعه دندانپزشکی ایران
3,3,دهمین همایش سالانه بانکداری الکترونیک
4,4,نمایشگاه و جشنواره هوش مصنوعی در گردشگری و تفر...


In [15]:
merged_df.isnull().sum()

index     0
Titles    0
dtype: int64

In [16]:
# use LabelEncoder for convert user(str) name to userId(int)
def iteraction_pre(df):
    df = df
    le = LabelEncoder()
    df["userId"] = le.fit_transform(df["نام و نام خانوادگی"])

    return df

In [17]:
df = iteraction_pre(interaction)

C:\Users\soheil\AppData\Local\Temp\ipykernel_16000\2212424727.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["userId"] = le.fit_transform(df["نام و نام خانوادگی"])


In [18]:
def list_to_string(row):
    return " ".join(row)

In [19]:
df_ohe = merged_df["Titles"].str.split(" ").reset_index().astype("str")

In [20]:
df_ohe["Titles"] = df_ohe["Titles"].apply(list_to_string)

In [21]:
le = LabelEncoder()
merged_df["ohe"] = le.fit_transform(df_ohe["Titles"])

In [22]:
def vectorized_text(df: pd.DataFrame):
    vectorized = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
    X = vectorized.fit_transform(merged_df["Titles"])

    feature_names = vectorized.get_feature_names_out()
    one_hot_df = pd.DataFrame(X.toarray(), columns=feature_names)

    dfs = pd.concat([df, one_hot_df], axis=1)
    dfs.drop(columns=["Titles"], inplace=True)

    return dfs

In [23]:
m_df = pd.DataFrame(merged_df["Titles"].unique())
m_df.columns = ["Titles"]
m_df

,Titles
0,کنسرت تست ( بدون انتخاب صندلی )
1,کنسرت تست ( با انتخاب صندلی )
2,کنگره بین‌المللی جامعه دندانپزشکی ایران
3,دهمین همایش سالانه بانکداری الکترونیک
4,نمایشگاه و جشنواره هوش مصنوعی در گردشگری و تفر...
...,...
80,کنسرت حامد زمانی
81,کنسرت احمد سلو
82,کنسرت نوشه ( ویژه بانوان )
83,تفریحات


In [24]:
dfs = vectorized_text(m_df)

In [25]:
def creat_X(df):

    M = df["userId"].nunique()

    N = df["بازدید"].nunique()

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))

    item_mapper = dict(zip(np.unique(df["بازدید"]), list(range(N))))

    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))

    item_inv_mapper = dict(zip(list(range(N)), np.unique(df["بازدید"])))

    user_index = [user_mapper[i] for i in df["userId"]]

    item_indx = [item_mapper[i] for i in df["بازدید"]]

    X = csr_matrix((df["زمان تعامل(تانیه)"], (user_index, item_indx)), shape=(M, N))

    return X, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper

In [26]:
X, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper = creat_X(df)

In [27]:
def find_similar_item(iter_name, X, item_mapper, item_inv_mapper, k, metrics="cosine"):

    X = X.T
    neighbours_ids = []

    iter_ind = item_mapper[iter_name]
    iter_vec = X[iter_ind]
    if isinstance(iter_vec, (np.ndarray)):
        iter_vec = iter_vec.reshape(1, -1)
    # use k+1 since kNN output includes the user recommender of interest
    knn = NearestNeighbors(n_neighbors=k + 1, algorithm="brute", metric=metrics)
    knn.fit(X)
    neighbours = knn.kneighbors(iter_vec, return_distance=False)
    for i in range(0, k):
        n = neighbours.item(i)
        neighbours_ids.append(item_inv_mapper[n])
    neighbours_ids.pop(0)
    return neighbours_ids

In [28]:
find_similar_item("تور دبی", X, item_mapper, item_inv_mapper, k=2, metrics="cosine")

['اخبار و مقالات']

In [29]:
def cosine_similioraty(
    dfs: pd.DataFrame, merge_pd, itraction, idx: str, n_recommendations: int = 1
):
    cosine_sim = cosine_similarity(dfs, dfs)
    iter_idx = dict(zip(merged_df["Titles"].unique(), list(itraction.index)))
    idx = iter_idx[idx]
    n_recommendations = n_recommendations
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1 : (n_recommendations + 1)]
    similar_item = [i[0] for i in sim_scores]
    recomended = merge_pd["Titles"].iloc[similar_item]
    recomended = recomended.to_list()

    return recomended

In [30]:
cosine_similarity(dfs, dfs).shape

(11413, 11413)

In [31]:
users_phone = interaction["شماره موبایل"].unique()

In [162]:
user_iter = {}
for i in users_phone:
    user_it = (
        interaction[interaction["شماره موبایل"] == i][["زمان تعامل(تانیه)", "بازدید"]]
        .sort_values(by="زمان تعامل(تانیه)", ascending=False)
        .iloc[:3]
    )
    user_it.columns = ["زمان تعامل(تانیه)", "بازدید"]
    # user_it.drop(index="index", inplace=True)
    user_it["بازدید"]
    idx = user_it["بازدید"].iloc[0]
    phone = str(i)
    phone = phone[:-2]
    iters = cosine_similioraty(dfs, merged_df, interaction, idx=idx)
    user_dict = {phone: iters}
    user_iter.update(user_dict)
user_iter

{'no': ['خدمات گردشگری'],
 '9101022210': ['تور قشم'],
 '9145236873': ['کنسرت نمایش کلنل'],
 '9101758962': ['کنگره بین\u200cالمللی جامعه دندانپزشکی ایران'],
 '9122070929': ['تور پوکت'],
 '9117803218': ['پارکینگ VIP']}

In [182]:
user_iter = {}
phone_list = []
recomend_df = pd.DataFrame()

for i in users_phone:
    user_it = (
        interaction[interaction["شماره موبایل"] == i][["زمان تعامل(تانیه)", "بازدید"]]
        .sort_values(by="زمان تعامل(تانیه)", ascending=False)
        .iloc[:2]
    )
    user_it.columns = ["زمان تعامل(تانیه)", "بازدید"]
    # user_it.drop(index="index", inplace=True)
    phone_list.append(i)
    user_it.columns = ["زمان تعامل(تانیه)", "بازدید"]
    user_it["بازدید"]
    idx = user_it["بازدید"].iloc[0]
    phone = str(i)
    phone = phone[:-2]
    iters = cosine_similioraty(dfs, merged_df, interaction, idx=idx)
    user_dict = {phone: iters}
    user_iter.update(user_dict)
pd.DataFrame(user_iter).T

,0
no,خدمات گردشگری
9101022210,تور قشم
9145236873,کنسرت نمایش کلنل
9101758962,کنگره بین‌المللی جامعه دندانپزشکی ایران
9122070929,تور پوکت
9117803218,پارکینگ VIP


In [191]:
df_list = []

for k in phone_list:
    pre = (
        interaction[interaction["شماره موبایل"] == k][["زمان تعامل(تانیه)", "بازدید"]]
        .sort_values(by="زمان تعامل(تانیه)", ascending=False)
        .iloc[:3]
    )
    df_dict = pre.to_dict(orient="list")  # Convert DataFrame to dictionary of lists
    df_list.append(df_dict)

recomend_df = pd.DataFrame(
    df_list, index=phone_list
)  # Create DataFrame from list of dictionaries
recomend_df.loc[:, "بازدید"]

none            [قوانین و مقررات, قوانین و مقررات, کنسرت تست (...
9101022210.0              [تور آنتالیا, تور آنتالیا, تور آنتالیا]
9145236873.0                                 [ایوان بند - خوشگله]
9101758962.0    [کارتهای E-Pass, پارک امیرگان ( Test ), پارک ا...
9122070929.0                                           [نمایشگاه]
9117803218.0    [دهمین همایش سالانه بانکداری الکترونیک, کنسرت ...
Name: بازدید, dtype: object

In [198]:
phone_dfs = {}

for k in phone_list:
    pre = interaction[interaction["شماره موبایل"] == k][
        ["زمان تعامل(تانیه)", "بازدید"]
    ].sort_values(by="زمان تعامل(تانیه)", ascending=False)
    df = pd.DataFrame(pre)  # Create DataFrame for each phone number
    phone_dfs[k] = df

# Accessing a DataFrame for a specific phone number:
# phone_dfs["123456789"]
pre

,زمان تعامل(تانیه),بازدید
28357,11.184,دهمین همایش سالانه بانکداری الکترونیک
27735,5.801,کنسرت تست ( با انتخاب صندلی )
28361,4.317,کنسرت احمد سلو
28365,4.289,کنسرت نمایش کلنل
29049,2.862,پخش آنلاین
28355,2.772,دهمین همایش سالانه بانکداری الکترونیک
29048,2.579,پخش آنلاین
28363,2.534,کنسرت نوشه ( ویژه بانوان )
28358,1.879,دهمین همایش سالانه بانکداری الکترونیک
28367,1.625,پخش آنلاین


In [187]:
interaction[interaction["شماره موبایل"] == k][
    ["زمان تعامل(تانیه)", "بازدید"]
].sort_values(by="زمان تعامل(تانیه)", ascending=False).iloc[:3]

,زمان تعامل(تانیه),بازدید
28357,11.184,دهمین همایش سالانه بانکداری الکترونیک
27735,5.801,کنسرت تست ( با انتخاب صندلی )
28361,4.317,کنسرت احمد سلو


In [173]:
user_it

,زمان تعامل(تانیه),بازدید
28357,11.184,دهمین همایش سالانه بانکداری الکترونیک
27735,5.801,کنسرت تست ( با انتخاب صندلی )


In [159]:
len(phone_list)

5

In [118]:
idxs = user_it["بازدید"].iloc[1]
type(idxs)

str

In [123]:
user_iter = {}
for i in users_phone:
    user_it = (
        interaction[interaction["شماره موبایل"] == i][["زمان تعامل(تانیه)", "بازدید"]]
        .max()
        .reset_index()
        .T
    )
    user_it.columns = ["زمان تعامل(تانیه)", "بازدید"]
    user_it.drop(index="index", inplace=True)
    user_it["بازدید"]
    idx = user_it["بازدید"].to_list()
    print(idx)

['کیوسکهای فروش آنلاین در مراکز تجاری']
['تور آنتالیا']
['ایوان بند - خوشگله']
['کنسرت']
['نمایشگاه']
['کنسرت نوشه ( ویژه بانوان )']


In [80]:
i

9117803218.0

In [72]:
user_it["بازدید"].to_list()

['کنسرت نوشه ( ویژه بانوان )']

In [61]:
interaction[interaction["شماره موبایل"] == i][
    ["زمان تعامل(تانیه)", "بازدید"]
].sort_values(by="زمان تعامل(تانیه)", ascending=False).iloc[:3]

,زمان تعامل(تانیه),بازدید
28357,11.184,دهمین همایش سالانه بانکداری الکترونیک
27735,5.801,کنسرت تست ( با انتخاب صندلی )
28361,4.317,کنسرت احمد سلو


In [40]:
cosine_similioraty(dfs, merged_df, interaction, idx="زانکو - گوشی خاموش")

['سالن اجرای صحنه ای 360']

In [33]:
iter_idx = dict(zip(merged_df["Titles"].unique(), list(interaction.index)))

In [34]:
cosine_sim = cosine_similarity(dfs, dfs)

In [35]:
def get_content_based_recommendations(title_string, n_recommendations=10):
    title = str(title_string)
    idx = iter_idx[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1 : (n_recommendations + 1)]
    similar_movies = [i[0] for i in sim_scores]
    print(f"Because you watched {title}:")
    print(merged_df["Titles"].iloc[similar_movies])

In [228]:
for i in phone_list:
    title = (
        interaction[interaction["شماره موبایل"] == i][["زمان تعامل(تانیه)", "بازدید"]]
        .sort_values(by="زمان تعامل(تانیه)", ascending=False)
        .iloc[:3]
    )
    title = title["بازدید"]

for k in range(len(title)):
    get_content_based_recommendations(title.iloc[k])

Because you watched دهمین همایش سالانه بانکداری الکترونیک:
11     پارکینگ VIP
12     پارکینگ VIP
38     پارکینگ VIP
97     پارکینگ VIP
98     پارکینگ VIP
99     پارکینگ VIP
118    پارکینگ VIP
149    پارکینگ VIP
160    پارکینگ VIP
192    پارکینگ VIP
Name: Titles, dtype: object
Because you watched کنسرت تست ( با انتخاب صندلی ):
69     نمایش کمدی ژیلت
70     نمایش کمدی ژیلت
71     نمایش کمدی ژیلت
72     نمایش کمدی ژیلت
101    نمایش کمدی ژیلت
127    نمایش کمدی ژیلت
196    نمایش کمدی ژیلت
236    نمایش کمدی ژیلت
285    نمایش کمدی ژیلت
373    نمایش کمدی ژیلت
Name: Titles, dtype: object
Because you watched کنسرت احمد سلو:
244    دانلود نرم افزار و قرارداد
295    دانلود نرم افزار و قرارداد
354    دانلود نرم افزار و قرارداد
426    دانلود نرم افزار و قرارداد
553    دانلود نرم افزار و قرارداد
576    دانلود نرم افزار و قرارداد
717    دانلود نرم افزار و قرارداد
833    دانلود نرم افزار و قرارداد
866    دانلود نرم افزار و قرارداد
886    دانلود نرم افزار و قرارداد
Name: Titles, dtype: object
